In [2]:
import openeo
import geopandas as gpd
import pandas as pd
from landuse_classification import *
from datetime import date

In [3]:
con = openeo.connect("https://openeo-dev.vito.be")

#### Target data 
LUCAS, 2018 (land use cover)

#### Input data
From S2: calculation of 7 indices (NDVI, NDMI, NDGI, ANIR, NDRE1, NDRE2, NDRE5) and keeping 2 bands (B06, B12)
From S1: VV, VH and VV/VH
For all of these, 10 features: p25, p50, p75, sd and 6 t-steps, with flexible range

#### Model
Random Forest, trained using custom hyperparameter, 70/30 split

In [4]:
## User options
## TODO: @Bart Convert this into HTML widgets!


## % of test data
train_test_split = 0.3

algorithm = "RandomForest"

hyperparams = {
    "num_trees": 250,
    "mtry": 3
}

## USE ONLY S1, ONLY S2 or both
feature_raster = "both" # s1, s2 or both

## Area of interest
aoi = gpd.read_file('lucas/test_aoi.geojson')

## Stratification layer
strat_layer = gpd.read_file("lucas/test_stratification.geojson") #None or a geojson

## Easy lookup to see what classes represent
# print(lookup_lucas["C10"])

## Custom legend
final_labels = {
    "Broadleaved woodland": ["C10"],
    "Coniferous woodland": ["C20"],
    "Mixed woodland": ["C30"],
    "Shrubland": ["D00", "D10", "D20"],
    "Grassland": ["E00", "E10", "E20", "E30"],
    "Cropland": ["B00", "B10", "B20", "B30","B40", "B50", "B70", "B80"],
    "Bare land": ["F00", "F10", "F20", "F40"],
    "Lichens and moss": ["F30"],
    "Built-up": ["A00", "A10", "A20", "A30"],
    "Inland water": ["G10", "G20", "G30"],
    "Sea and ocean": ["G40"],
    "Glaciers and permanent snow": ["G50"],
    "Wetlands": ["H00", "H10", "H20"]
}

## Cleaning & filtering operations on dataset
include_mixed_pixels = False
max_nr_samples = None
start_date = date(2018,3,15)
end_date = date(2018,10,31)
stepsize_s2 = 10
stepsize_s1 = 12

In [5]:
## TODO: @Bart use extended polygon LUCAS set https://essd.copernicus.org/articles/13/1119/2021/ and then extract points
data = gpd.read_file("https://artifactory.vgt.vito.be/auxdata-public/openeo/lucas.gpkg",mask=aoi)

if data.empty:
    raise ValueError("Your masked area is located outside of Europe or so small that no training data can be found within it")

In [20]:
## TODO: @Bart only keep label + geometry and make sure label matches general labels ([0:2] from label)
y = data[["LC1", "geometry"]]
y2 = y.buffer(0.0001)

<ipython-input-20-7843e3782b1e>:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y2 = y.buffer(0.0001)


In [32]:
y3 = y2.to_json()
y4 = y.to_json()

In [ ]:
if strat_layer is None:
    strat_layer = aoi

In [16]:
def load_lc_features(feature_raster, aoi, start_date, end_date, stepsize_s2=10, stepsize_s1=12):
    provider = "terrascope"
    
    idx_dekad = sentinel2_features(start_date, end_date, connection, provider, processing_opts={}, sampling=True, stepsize=stepsize_s2)
    idx_features = compute_statistics(idx_dekad, start_date, end_date, stepsize=stepsize_s2)

    s1_dekad = sentinel1_features(start_date, end_date, connection, provider, processing_opts={}, orbitDirection="ASCENDING", sampling=True, stepsize=stepsize_s1)
    s1_dekad = s1_dekad.resample_cube_spatial(idx_dekad)
    s1_features = compute_statistics(s1_dekad, start_date, end_date, stepsize=stepsize_s1)

    features = idx_features.merge_cubes(s1_features)

    if feature_raster == "s1":
        return s1_features, features.metadata.band_names
    elif feature_raster == "s2":
        return idx_features, idx_features.metadata.band_names
    else:
        return features, features.metadata.band_names
    
from openeo_classification.connection import connection

In [51]:
# for stratum in strat_layer.iterfeatures():
# final_training_data = gpd.clip(data, gpd.GeoDataFrame(stratum))
    ## TODO: @Bart load_features spatial_extents toevoegen
features, feature_list = load_lc_features(feature_raster, y, start_date, end_date, stepsize_s2, stepsize_s1)
import json
    ## TODO: @Bart testen tot aan hier!
# X = features.aggregate_spatial(geometries=eval(y4), reducer="mean")
# X.download("aggspatialtest.csv", format="CSV")
sampled_features = features.filter_spatial(json.loads(y3))
job = sampled_features.execute_batch(
        title="Test aggregate spatial",
        description="Test aggregate spatial",
        out_format="netCDF",
        sample_by_feature=True)
results = job.get_results()
results.download_files("aggspatialtest_traditional")
    ## dit testen tot hier

# X_fin = X.execute()
# from openeo.rest.conversions import timeseries_json_to_pandas
# X_finfin = timeseries_json_to_pandas(timeseries)
# print(X_finfin)

    ## TODO ? load_vectorcube, wellicht niet nodig gezien we alleen hoeven clippen naar een spatial extent. Load_files?
#     target = connection.load_vectorcube("x.geojson")
    
    ## TODO: fit_class_random_forest werkend krijgen voor X en y als inputs
#     ml_model = con.fit_class_random_forest(predictors=X, target=y, training=100, **hyperparams)
#     ml_model = ml_model.save_ml_model().start_and_wait().get_result()
    

Authenticated using refresh token.
Authenticated using refresh token.
0:00:00 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': send 'start'
0:00:34 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:00:41 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:00:48 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:00:56 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:01:07 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:01:20 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:01:37 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:01:57 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:02:22 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': queued (progress N/A)
0:02:53 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': running (progress N/A)
0:03:31 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb': running (progress N/A)
0:04:19 Job 'b6593759-c7f1-4773-a487-9f69bb4636bb

[WindowsPath('aggspatialtest_traditional/openEO_0.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_1.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_10.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_11.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_12.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_13.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_14.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_15.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_16.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_17.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_18.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_19.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_2.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_20.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_21.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_22.nc'),
 WindowsPath('aggspatialtest_traditional/openEO_23.nc'),
 WindowsPath('aggspatialtest_tradi

In [ ]:
## Final inference: 2-3 tiles, 2-3 different scenarios
for stratum in strat_layer.iterfeatures():
    con.load_ml_model('batch_job_id')
    features, feature_list = load_features(feature_raster, stratum, start_date, end_date, stepsize_s2, stepsize_s1)

    def predict_rf(x: ProcessBuilder):
        ## TODO: bedenken hoe dit eruit gaat zien
        return x.predict_random_forest(features)

    y_pred = datacube.reduce_dimension(dimension="bands", process=predict_rf)

## Daarna verschillende predicties door verschillende strata weer terug mergen in 1 beeld ?
    
from sklearn.metrics import precision_recall_fscore_support
prec, rec, fscore, sup = precision_recall_fscore_support(y_test,y_pred)

In [ ]:
## Nice to have: custom rules toevoegen
#     ## Applying custom rules to the classification
#     def rule_glaciers(x: ProcessBuilder):
#         if x.is_nodata():
#             return array_create(data=[65534]*120)
#         return x    
#     datacube.apply_dimension(dimension="t", process=rule_glaciers)